In [ ]:
%run ./define_tables

: 

In [0]:
from delta.tables import DeltaTable

silver_london_local_authority_data = spark.table(silver_london_authorities_table)

silver_data = spark.table('`catalog-dev-uks-corecdp-001`.`schema-dev-uks-corecdp-silver-001`.environmental_data')

location_environment_data = (
    silver_data
    .join(silver_london_local_authority_data, on='site')
)

if not spark.catalog.tableExists(gold_table):
    (location_environment_data
        .limit(0)                     # no rows, only schema
        .write
        .format("delta")
        .saveAsTable(gold_table))

gold_table = DeltaTable.forName(spark, gold_table)
(
    gold_table.alias("t")
    .merge(
        location_environment_data.alias("s"),
        """
        t.site = s.site
        AND t.species = s.species
        AND t.reading_datetime = s.reading_datetime
        """
    )
    .whenNotMatchedInsertAll()
    .execute()
)

gold_table.toDF().show(5)
# count rows in gold table 
display(gold_table.toDF().count())



In [0]:
base_query = """
SELECT
    site, 
    DATE(reading_datetime) AS reading_date,  -- truncate timestamp to day
    AVG(value)
FROM 
    `catalog-dev-uks-corecdp-001`.`schema-dev-uks-corecdp-gold-001`.london_environment_data
WHERE species = '{species}'
AND reading_datetime >= '{start_date}'
AND reading_datetime <= '{end_date}'
GROUP BY
    site,
    DATE(reading_datetime)
ORDER BY
    site,
    reading_date
"""


In [0]:
query = base_query.format(
    species="NOX", 
    start_date="2024-01-01", 
    end_date="2024-12-31"
)
df = spark.sql(query)
display(df)


In [0]:
query = base_query.format(
    species="NO2", 
    start_date="2024-01-01", 
    end_date="2024-12-31"
)
df = spark.sql(query)
display(df)